In [644]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import tree
import matplotlib.pyplot as plt


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

kaggle/input/.DS_Store
kaggle/input/titanic/test.csv
kaggle/input/titanic/train.csv


In [645]:
train_data = pd.read_csv("kaggle/input/titanic/train.csv")
train_data.head()

test_data = pd.read_csv("kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [646]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder  


device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')


class Model(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Model, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.ReLU(),
            nn.Linear(output_dim, 1),
            nn.Sigmoid(),
        )
    
    def forward(self, x):
        out = self.model(x)
        return out

model = Model(5, 1024)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

In [647]:
def prep_data(data):
    X_Train = data.drop(["Name"], axis=1)
    X_Train = X_Train.drop(["Ticket"], axis=1)
    X_Train = X_Train.drop(["Cabin"], axis=1)
    X_Train = X_Train.drop(["Embarked"], axis=1)
    X_Train = X_Train.drop(["PassengerId"], axis=1)
    X_Train = X_Train.drop(["Fare"], axis=1)
    X_Train["Age"] = X_Train["Age"].fillna(X_Train["Age"].mean())
    try:
        X_Train = X_Train.drop(["Survived"], axis=1)
    except:
        pass

    le = LabelEncoder()
    X_Train["Sex"] = le.fit_transform(X_Train["Sex"])
    #X_Train["Age"] = le.fit_transform(X_Train["Age"])
    return X_Train

X_Train = prep_data(train_data)
X_Test = prep_data(test_data)
y = train_data["Survived"]

X_Train.head()

,Pclass,Sex,Age,SibSp,Parch
0,3,1,22.0,1,0
1,1,0,38.0,1,0
2,3,0,26.0,0,0
3,1,0,35.0,1,0
4,3,1,35.0,0,0


In [648]:

epoch = 500
X_Train = torch.from_numpy(X_Train.to_numpy()).float()
X_Test = torch.from_numpy(X_Test.to_numpy()).float()
y = torch.from_numpy(y.to_numpy()).float()

train_data_loader = DataLoader(TensorDataset(X_Train, y), batch_size=4, shuffle=True)
test_data_loader = DataLoader(TensorDataset(X_Train, y), batch_size=1, shuffle=True)

print(X_Train)

for n in range(epoch):
    for x, y_test in train_data_loader:
        optimizer.zero_grad()
        y_pred = model(x)
        loss = nn.functional.mse_loss(y_pred, y_test)
        loss.backward()
        optimizer.step()

    print("Epoch: ", n, "Loss: ", loss.item())

tensor([[ 3.0000,  1.0000, 22.0000,  1.0000,  0.0000],
        [ 1.0000,  0.0000, 38.0000,  1.0000,  0.0000],
        [ 3.0000,  0.0000, 26.0000,  0.0000,  0.0000],
        ...,
        [ 3.0000,  0.0000, 29.6991,  1.0000,  2.0000],
        [ 1.0000,  1.0000, 26.0000,  0.0000,  0.0000],
        [ 3.0000,  1.0000, 32.0000,  0.0000,  0.0000]])
Epoch:  0 Loss:  0.2670636475086212
Epoch:  1 Loss:  0.14677417278289795
Epoch:  2 Loss:  0.27308329939842224
Epoch:  3 Loss:  0.22301054000854492


/var/folders/s0/xcg0d2_s6tg_2v3r53qlxyl40000gn/T/ipykernel_1681/2265104275.py:15: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = nn.functional.mse_loss(y_pred, y_test)
/var/folders/s0/xcg0d2_s6tg_2v3r53qlxyl40000gn/T/ipykernel_1681/2265104275.py:15: UserWarning: Using a target size (torch.Size([3])) that is different to the input size (torch.Size([3, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = nn.functional.mse_loss(y_pred, y_test)


Epoch:  4 Loss:  0.2740204632282257
Epoch:  5 Loss:  0.23996655642986298
Epoch:  6 Loss:  0.2247389405965805
Epoch:  7 Loss:  0.2236030399799347
Epoch:  8 Loss:  0.2308950424194336
Epoch:  9 Loss:  0.2238209992647171
Epoch:  10 Loss:  0.22854161262512207
Epoch:  11 Loss:  0.22547949850559235
Epoch:  12 Loss:  0.0845305547118187
Epoch:  13 Loss:  0.23388749361038208
Epoch:  14 Loss:  0.12703879177570343
Epoch:  15 Loss:  0.2260761857032776
Epoch:  16 Loss:  0.07092224061489105
Epoch:  17 Loss:  0.2885846197605133
Epoch:  18 Loss:  0.23259319365024567
Epoch:  19 Loss:  0.2519058287143707
Epoch:  20 Loss:  0.23121346533298492
Epoch:  21 Loss:  0.22519728541374207
Epoch:  22 Loss:  0.22259241342544556
Epoch:  23 Loss:  0.2862231731414795
Epoch:  24 Loss:  0.26983800530433655
Epoch:  25 Loss:  0.15925420820713043
Epoch:  26 Loss:  0.06564989686012268
Epoch:  27 Loss:  0.2280551791191101
Epoch:  28 Loss:  0.2508012354373932
Epoch:  29 Loss:  0.23217371106147766
Epoch:  30 Loss:  0.4007066488

In [649]:
num_right = 0
for x, y_test in test_data_loader:
    y_pred = model(x)
    ypred = int(y_pred.round().item())
    if ypred == y_test.item():
        num_right += 1

print("Accuracy: ", num_right / len(train_data))

Accuracy:  0.621773288439955


In [650]:
predictions = []
for n in X_Test:
    predictions.append(int(model(n).round().item()))

print(predictions)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 